# Final Project

## Title: Automatic Maze Generation with Various Algorithms

### Author: Vinny Sun



Feedback on Abstract:

This sounds like a very interesting project! Below are recommendations how to make the notebook reader-friendly:

- to be successful, the project should focus in on only one (or at most two) topics; 
- it would help to start with simple examples that can be computed by hand;
- describe the basic idea behind your algorithm in words;
- where possible, show the functioning of subcomponents of your algorithm, in addition to the functioning of the whole thing;
- the nice thing about Python is that when you look at the docs, every function comes with its source file (look to the right for the link [source]), so you can inspect it, play around with it, use it to explain it etc; 
- give some relevant citations;

Mazes can be generated algorithmically in many ways. We will define a maze as a [square lattice graph](https://en.wikipedia.org/wiki/Lattice_graph) with some edges removed such that there is a path from a starting point to an ending point. It would also be nice if this maze was interesting enough to be "fun" for the maze solver. We will look through various algorithms to 
- remove edges from a square lattice graph
- create mazes

Some of these mazes will be [perfect mazes](https://professor-l.github.io/mazes). Inspiration for the ideas and algorithms come from [here](https://weblog.jamisbuck.org/2011/2/7/maze-generation-algorithm-recap) and [here](https://en.wikipedia.org/wiki/Maze_generation_algorithm) with ideas for the implementation coming from [here](https://medium.com/@msgold/using-python-to-create-and-solve-mazes-672285723c96).


---
We start by importing necessary packages

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

From [here](https://en.wikipedia.org/wiki/Maze_generation_algorithm), a maze can be seen as subgraph of a connected planar graph where the nodes are cells and the edges are possible wall sites and there is a challenging route from one node to another. Thus,